In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch
from datasets import Dataset
import pandas as pd
import os

# Define directories and file paths
OUTPUT_DIR = "results"
LOG_DIR = "logs"
TRAIN_FILE = "dataset/train.txt"  # Path to your training data file
VAL_FILE = "dataset/val.txt"  # Path to your validation data file
TEST_FILE = "dataset/test.txt"  # Path to your test data file

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-360M-Instruct")
# for multiple GPUs install accelerate and do `model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")`
model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-360M-Instruct")

# Enable gradient checkpointing to save memory
model.gradient_checkpointing_enable()

# Resize token embeddings if needed
model.resize_token_embeddings(len(tokenizer))

# Clear GPU cache and set device to CUDA if available
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to count tokens in a file
def count_tokens(file_path, tokenizer):
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
    tokens = tokenizer.tokenize(text)
    return len(tokens)

# Count tokens in training and validation files
train_token_count = count_tokens(TRAIN_FILE, tokenizer)
eval_token_count = count_tokens(VAL_FILE, tokenizer)
print(f"Train Tokens: {train_token_count}")
print(f"Validation Tokens: {eval_token_count}")

# Training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=1,  # Reduced batch size for limited GPU memory
    per_device_eval_batch_size=1,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir=LOG_DIR,
    fp16=True,  # Enable mixed precision training for GPU efficiency
    bf16=False,  # Disable bf16 as it's not supported on all GPUs
    gradient_accumulation_steps=4,  # Accumulate gradients to simulate larger batch sizes
)

# Function to load chatbot data
def load_chatbot_data(file_path):
    """Load and preprocess chatbot data from the given text file."""
    conversations = []
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
        user_input, bot_response = None, None
        for line in lines:
            if line.startswith("User:"):
                user_input = line.replace("User:", "").strip()
            elif line.startswith("Bot:"):
                bot_response = line.replace("Bot:", "").strip()
                if user_input and bot_response:
                    conversations.append({"input": user_input, "output": bot_response})
                    user_input, bot_response = None, None
    return pd.DataFrame(conversations)

# Prepare datasets
df_train = load_chatbot_data(TRAIN_FILE)
df_val = load_chatbot_data(VAL_FILE)
dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)

# Tokenization function
def tokenize_function(examples):
    inputs = [f"{inp} {out}" for inp, out in zip(examples["input"], examples["output"])]
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )
    model_inputs["labels"] = model_inputs["input_ids"].clone()  # Labels are the same as input_ids for causal LM
    return model_inputs

# Apply tokenization
tokenized_train = dataset_train.map(tokenize_function, batched=True)
tokenized_val = dataset_val.map(tokenize_function, batched=True)

# Data collator for causal language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Masked language modeling is not used for causal LM
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train the model
trainer.train()

# Save the trained model and tokenizer
model.save_pretrained(os.path.join(OUTPUT_DIR, "test2_cb"))
tokenizer.save_pretrained(os.path.join(OUTPUT_DIR, "test2_cb"))

Train Tokens: 7144
Validation Tokens: 1580


Map:   0%|          | 0/96 [00:00<?, ? examples/s]

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

C:\Users\wongq\AppData\Local\Temp\ipykernel_5352\4143569196.py:105: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss
1,No log,1.690580
2,No log,1.647575
3,No log,1.645658
4,No log,1.651312
5,No log,1.669912
6,No log,1.692904
7,No log,1.708447
8,No log,1.731985
9,No log,1.743749
10,No log,1.749091


('results\\test2_cb\\tokenizer_config.json',
 'results\\test2_cb\\special_tokens_map.json',
 'results\\test2_cb\\vocab.json',
 'results\\test2_cb\\merges.txt',
 'results\\test2_cb\\added_tokens.json',
 'results\\test2_cb\\tokenizer.json')

In [5]:
model = AutoModelForCausalLM.from_pretrained("results/test2_cb", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("results/test2_cb", device_map="auto")

model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 960, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=960, out_features=960, bias=False)
          (k_proj): Linear(in_features=960, out_features=320, bias=False)
          (v_proj): Linear(in_features=960, out_features=320, bias=False)
          (o_proj): Linear(in_features=960, out_features=960, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=960, out_features=2560, bias=False)
          (up_proj): Linear(in_features=960, out_features=2560, bias=False)
          (down_proj): Linear(in_features=2560, out_features=960, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((960,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((960,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((960,), eps=1e-05)
    (rotary_emb)

In [3]:
def chatbot_response(prompt, max_length=100):
    # Tokenize input prompt
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Generate response
    outputs = model.generate(**inputs, max_length=max_length)

    # Decode the generated text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [6]:
if __name__ == "__main__":
    print("Chatbot is ready! Type 'exit' to stop.")
    while True:
        user_input = input("User: ")
        if user_input.lower() == "exit":
            break
        response = chatbot_response(user_input)
        print(f"User: {user_input}")
        print(f"Bot: {response}")

Chatbot is ready! Type 'exit' to stop.
User: what is depression?
Bot: what is depression? Depression is a serious mental health condition that can affect how you feel, think, and behave. It can cause feelings of sadness, hopelessness, and loss of interest in things you once enjoyed. Depression can also impact your energy, motivation, and ability to function. If you think you or someone you know might be experiencing depression, it's important to talk to a healthcare provider. They can help you understand what you're going through and develop a treatment plan. What do you think?
User: i think my friend has depression. how can i help?
Bot: i think my friend has depression. how can i help? I want to support my friend, but I don't know where to start. I want to help them feel less alone. I think depression can be tough to talk about, but its important to listen and offer support. You can start by letting your friend know theyre not alone and that youre there for them. You can also encou